### Install Dependencies

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn
!pip install statsmodels

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

### Read Data into pandas df

In [ ]:
data = pd.read_csv('example_prices.csv', parse_dates=['dates'], index_col='dates')

In [ ]:
# Extract the last 400 data points
last_400 = data.tail(400)

past = last_400.iloc[:199].copy()  # First 210 points
last_20 = past.iloc[-20:].copy()   # Last 20 points of past
future = last_400.iloc[199:300] .copy() # Next 200 points (from index 190 onward)

## Trend Line (by regression) for the past 200 days

In [ ]:

past.loc[:, 'time'] = (past.index - past.index[0]).days
last_20.loc[:, 'time'] = (last_20.index - last_20.index[0]).days
future['time'] = (future.index - past.index[0]).days  # Ensure this is created

l = last_20[['time']]

X = past[['time']]  # Independent variable (time)
y = past['diamonds']  # Dependent variable (actual data)

# Fit a linear regression model
model = LinearRegression()
model.fit(X, y)

trend_line = model.predict(X)

# **Use the same model to extend the past trend line**
future['past_trend'] = model.predict(future[['time']])


## Find MAX and MIN for 200 to form belinger bounds

In [ ]:
# Find the maximum point in past
if not past['diamonds'].dropna().empty:
    max_index = past['diamonds'].idxmax()
    max_time = (max_index - last_400.index[0]).days
    max_value = past.loc[max_index, 'diamonds']

    # Calculate the equation of the new line: y = slope * (x - max_time) + max_value
    slope = model.coef_[0]
    max_line = slope * (X['time'] - max_time) + max_value

# Find the minimum point in past
if not past['diamonds'].dropna().empty:
    min_index = past['diamonds'].dropna().idxmin()
    min_time = (min_index - last_400.index[0]).days
    min_value = past.loc[min_index, 'diamonds']

    # Calculate the equation of the new line: y = slope * (x - min_time_past) + min_value_past
    min_line_past = slope * (X['time'] - min_time) + min_value

## Do the same as above but for last 20 days

In [ ]:
#TODO
# Find the maximum point in last_20
# Ensure last_20 has a time column
last_20['time'] = (last_20.index - last_20.index[0]).days

# Find the maximum point in last_20
if not last_20['diamonds'].dropna().empty:
    max_index_20 = last_20['diamonds'].idxmax()
    max_time_20 = (max_index_20 - last_20.index[0]).days  # FIXED
    max_value_20 = last_20.loc[max_index_20, 'diamonds']

    # Compute trend line from max point
    slope = model.coef_[0]
    max_line_20 = slope * (last_20['time'] - max_time_20) + max_value_20  # FIXED

# Find the minimum point in last_20
if not last_20['diamonds'].dropna().empty:
    min_index_20 = last_20['diamonds'].idxmin()
    min_time_20 = (min_index_20 - last_20.index[0]).days  # FIXED
    min_value_20 = last_20.loc[min_index_20, 'diamonds']

    # Compute trend line from min point
    min_line_20 = slope * (last_20['time'] - min_time_20) + min_value_20  # FIXED


## Adjusting trend line based on last 20 days averages

In [ ]:
weight = 20
intercept_shift = (-min_value_20 + max_value_20) / weight
trend_line_adjusted = model.predict(X) + intercept_shift

In [ ]:
crossing_index = None
for i in range(len(future)):
    if future.iloc[i]['diamonds'] > future.iloc[i]['past_trend']:  # Change condition for downward crossing
        crossing_index = future.index[i]
        break  # Stop at the first crossing point

if crossing_index is not None:
    print(f"The trend line is crossed at index: {crossing_index}")
else:
    print("No crossing detected in the given future points.")

In [ ]:
plt.figure(figsize=(10, 6))

# Plot a vertical line at the midpoint
x_mid = last_400.index[len(last_400) // 2]  # Middle point of the time index
plt.axvline(x=x_mid, color='red', linestyle='--', linewidth=2, label='Midpoint Vertical Line')


# Plot the actual data and trend line
plt.plot(past.index, y, label='Actual Data (Last 200 Points)', color='blue')


plt.plot(past.index, trend_line, label='Trend Line', color='orange', linestyle='--')
plt.plot(past.index, trend_line_adjusted, label='Trend Line', color='yellow', linestyle='--')

full_time = np.concatenate((past['time'], future['time']))[:, np.newaxis]  # Full time range

full_index = past.index.union(future.index)
# Extend the trend line
full_trend = model.predict(full_time)  # Predict over full range
plt.plot(full_index, full_trend, label='Extended Trend Line', color='black', linestyle='dashed')


# Plot actual data over last_400
plt.plot(future.index, future['diamonds'], label='Actual Data (Last 400 Points)', color='blue', alpha=0.5)


# Plot the new lines if they were calculated
if 'max_line' in locals():
    plt.plot(past.index, max_line, label='Line from Max Point', color='red', linestyle=':')
if 'min_line_past' in locals():
    plt.plot(past.index, min_line_past, label='Line from Min Point', color='green', linestyle=':')

if 'max_line_20' in locals():
    plt.plot(last_20.index, max_line_20, label='Line from Max 20', color='red')
if 'min_line_20' in locals():
    plt.plot(last_20.index, min_line_20, label='Line from Min 20', color='green')

# Add labels, title, and legend
plt.title('Actual Data vs Trend Line with Lines from Max and Min Points')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

# Ensure 'time' exists in both past and future
past['time'] = (past.index - past.index[0]).days
future['time'] = (future.index - past.index[0]).days

# Full time range for extending lines
full_index = past.index.union(future.index)
full_time = np.concatenate((past['time'], future['time']))[:, np.newaxis]

# Fit & extend trend line
X = past[['time']]
y = past['diamonds']
model = LinearRegression()
model.fit(X, y)
full_trend = model.predict(full_time)  # Extended trend

# Extend adjusted trend line
if 'trend_line_adjusted' in locals():
    adjusted_model = LinearRegression().fit(past[['time']], trend_line_adjusted)
    full_adjusted_trend = adjusted_model.predict(full_time)
else:
    full_adjusted_trend = None

# Extend max line
if 'max_line' in locals():
    max_model = LinearRegression().fit(past[['time']], max_line)
    full_max_line = max_model.predict(full_time)
else:
    full_max_line = None

# Extend min line
if 'min_line_past' in locals():
    min_model = LinearRegression().fit(past[['time']], min_line_past)
    full_min_line = min_model.predict(full_time)
else:
    full_min_line = None

# Extend max line from last 20 points
if 'max_line_20' in locals():
    max20_model = LinearRegression().fit(last_20[['time']], max_line_20)
    full_max20_line = max20_model.predict(full_time)
else:
    full_max20_line = None

# Extend min line from last 20 points
if 'min_line_20' in locals():
    min20_model = LinearRegression().fit(last_20[['time']], min_line_20)
    full_min20_line = min20_model.predict(full_time)
else:
    full_min20_line = None

# Plot everything
x_mid = last_400.index[len(last_400) // 2]
plt.axvline(x=x_mid, color='red', linestyle='--', linewidth=2, label='Midpoint Vertical Line')

# Past & future actual data
plt.plot(past.index, past['diamonds'], label='Past Data', color='blue')
plt.plot(future.index, future['diamonds'], label='Future Data', color='purple')

# Extended lines
plt.plot(full_index, full_trend, label='Extended Trend Line', color='black', linestyle='dashed')
if full_adjusted_trend is not None:
    plt.plot(full_index, full_adjusted_trend, label='Extended Adjusted Trend', color='yellow', linestyle='--')
if full_max_line is not None:
    plt.plot(full_index, full_max_line, label='Extended Max Line', color='red', linestyle=':')
if full_min_line is not None:
    plt.plot(full_index, full_min_line, label='Extended Min Line', color='green', linestyle=':')

plt.title('Actual Data vs Fully Extended Trend & Reference Lines')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
crossing_index = []
for i in range(len(future)):
    if future.iloc[i]['diamonds'] > future.iloc[i]['past_trend']:  # Change condition for downward crossing
        crossing_index.append(future.index[i])

for c in crossing_index:
    if c is not None:
        print(f"The trend line is crossed at index: {c}")